<a href="https://colab.research.google.com/github/farrukhrashid1997/Basic-Ecommerce-Application/blob/master/CIFAR_10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.datasets import cifar10
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
#loading the cifar10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 13s 0us/step


In [0]:
#Creating the configuration of the VGG model
#The main point here is that 3x3 filters are used since they will act as 5x5 filters, reducing the no of hyperparameters.
#This way we can introduce more non-linearity preventing overfitting.
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:] , activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding = 'same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding = 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))

model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 1)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [0]:
#I have used an imagedatagenerator, this basically iterates over the training data and flips some of them horizontally, changes the zoom range.
#This is done usually to make the model better, prevent it from overfitting.
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    dtype = 'float32',
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

#This generator is used as a validation generator or test generator
test_datagen = ImageDataGenerator(rescale=1./255, 
                                  dtype = 'float32')


validation_generator = test_datagen.flow(
    x_test, y_test, batch_size = 32)



train_datagen.fit(x_train)

In [0]:
train_generator = train_datagen.flow(
    x_train,
    y_train,
    batch_size=100)


#Usually more than 80% is achieved if 100 epochs were run. I wanted to try loading ResNet weights and try then. I didnt due to internet problems.
model.fit_generator(train_generator,
                    epochs = 10,
                    validation_data=validation_generator,
                    workers = 4
                    )

Epoch 1/10
500/500 [==============================] - 767s 2s/step - loss: 2.0001 - accuracy: 0.2678 - val_loss: 1.3747 - val_accuracy: 0.4320
Epoch 2/10
500/500 [==============================] - 711s 1s/step - loss: 1.4583 - accuracy: 0.4743 - val_loss: 1.2077 - val_accuracy: 0.5797
Epoch 3/10
500/500 [==============================] - 710s 1s/step - loss: 1.2094 - accuracy: 0.5710 - val_loss: 1.1106 - val_accuracy: 0.6043
Epoch 4/10
500/500 [==============================] - 744s 1s/step - loss: 1.0559 - accuracy: 0.6306 - val_loss: 0.7550 - val_accuracy: 0.6807
Epoch 5/10
500/500 [==============================] - 746s 1s/step - loss: 0.9555 - accuracy: 0.6692 - val_loss: 1.3230 - val_accuracy: 0.6707
Epoch 6/10
500/500 [==============================] - 755s 2s/step - loss: 0.8864 - accuracy: 0.6952 - val_loss: 1.1106 - val_accuracy: 0.6953
Epoch 7/10
500/500 [==============================] - 770s 2s/step - loss: 0.8391 - accuracy: 0.7104 - val_loss: 0.8593 - val_accuracy: 0.7253

In [0]:
#I have also saved the weights, we can recreate the model above and load the weights. 
model.save_weights('/content/Lab11.h5')

In [0]:
acc = model.evaluate_generator(validation_generator)

In [0]:
print("The accuracy of the model is ",  acc[1]*100)

The accuracy of the model is  73.36999773979187
